# D. CRPSS calculation

Since the seasonal forecasts are comprised of ensemble members, generally applied simple evaluation methods such as RMSE (Root Mean Squared Error) and R2 have limitations. 
Thus, in many studies, CRPS or CRPSS are adopted to measure the skill of ensembled datasets.
This code enables you to calculate those indices automatically.

### 1. Import Libraries

Let's start by importing the necessary libraries (🚨 in order to run the code like in the cell below, place the mouse pointer in the cell, then click on “run cell” button above or press shift + enter).

In [2]:
import cdsapi
import netCDF4
from netCDF4 import num2date, Dataset
import numpy as np
import os
import pandas as pd
import hydrostats.ens_metrics as em
import datetime

### 2. Basic settings

Again, we have to make basic settings to calculate CRPS and CRPSS.

In [4]:
originating_centre = 'ECMWF'
variable = 'tp'               # Information that you want to calculate areal mean
catch_name = 'cj'             # What is the name of your catchment?
maxleadtime = 6               # Number of months that you want to extract
start_year = 2011                # CRPS/CRPSS calculation start year
end_year = 2020                # CRPS/CRPSS calculation end year
start_month = 1                  # CRPS/CRPSS calculation start month
end_month = 12                 # CRPS/CRPSS calculation end month
days = {1:31,2:28,3:31,4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30,12:31}

inpath = './assessment/' + str(originating_centre.upper())+ '/daily/'
bcpath = './assessment/' + str(originating_centre.upper())+ '/biascorrection/'
calcpath = './assessment/' + str(originating_centre.upper())+ '/calculation/'

### 3. Seasonal forecasts CRPS calculation

###   3.1 Rearrange Datasets by lead time (Seasonal forecasts)

Basically, CRPS/CRPSS are calculated according to lead time. Therefore, we need to collect them into a single file for each lead time. This code enable to collect every data having same lead time. 

Considering that we have 2 types datasets, before and after bias correction, <font color = 'red'> You don't need to repeat this process 2 times, since automatically this code perforems for both.</font> 

In [16]:
# Before bias correction
bc_type = 'before'
# Allocation of inputfile path and tail annotation

for leadtime in range(1,maxleadtime+1):
    for years in range(start_year,end_year+1):
        for months in range(start_month, end_month+1):
            df = pd.read_csv(inpath + str(catch_name) + '_' + str(years) + '_' + str(months) + '_' + str(originating_centre.lower())+ '_' + str(variable) + '_mean.csv')
            df2=df.groupby(by=['leadtime']).sum().cumsum()
            df2['date'] = str(years) + '_' + str(months)
            col1=df2.columns[-1:].to_list()
            col2=df2.columns[:-1].to_list()
            new_col=col1+col2
            df3=df2[new_col]
            temp = pd.DataFrame(df3.loc[leadtime]).T
            if years == start_year and months ==start_month:
                temp1 = temp
            else :
                pass
            temp1 = temp1.append(temp, ignore_index = True)
    temp1 = temp1.iloc[1:]
    temp1['mean2'] = temp1['mean']
    temp1['obs2'] = temp1['obs']
    temp1=temp1.drop(['mean','obs'], axis=1)
    temp1.rename(columns={'mean2':'mean', 'obs2':'obs'}, inplace=True)
    temp1.set_index('date', inplace=True)
    temp1.to_csv(calcpath + str(catch_name) + '_' + str(leadtime) + '_' + bc_type + '_bc.csv')

# After bias correction
bc_type = 'after'
# Allocation of inputfile path and tail annotation

for leadtime in range(1,maxleadtime+1):
    for years in range(start_year,end_year+1):
        for months in range(start_month, end_month+1):
            df = pd.read_csv(bcpath + str(catch_name) + '_' + str(years) + '_' + str(months) + '_' + str(originating_centre.lower())+ '_' + str(variable) + '_mean_bc.csv')
            df2=df.groupby(by=['leadtime']).sum().cumsum()
            df2['date'] = str(years) + '_' + str(months)
            col1=df2.columns[-1:].to_list()
            col2=df2.columns[:-1].to_list()
            new_col=col1+col2
            df3=df2[new_col]
            temp = pd.DataFrame(df3.loc[leadtime]).T
            if years == start_year and months ==start_month:
                temp1 = temp
            else :
                pass
            temp1 = temp1.append(temp, ignore_index = True)
    years = years+1
    temp1 = temp1.iloc[1:]
    temp1['mean2'] = temp1['mean']
    temp1['obs2'] = temp1['obs']
    temp1=temp1.drop(['mean','obs'], axis=1)
    temp1.rename(columns={'mean2':'mean', 'obs2':'obs'}, inplace=True)
    temp1.set_index('date', inplace=True)
    temp1.to_csv(calcpath + str(catch_name) + '_' + str(leadtime) + '_' + bc_type + '_bc.csv')

print('Successful')

Successful


#### 3.2 CRPS calculation (Seasonal forecasts)

CRPS is a measure of how good forecasts are in matching observed outcomes considering each ensemble. It is a quadratic measure of the difference between the forecast cumulative distribution function (CDF) and the reference dataset of the observation (Zamo and Naveau, 2017). The CRPS is thus calculated as

$$ CRPS= \int [F(x) - H(x > y)]^2 dx $$

where F(x) represents the cumulative distribution of seasonal forecasts, y is observed precipitation, H is called the indicator function which is equals to 1 when x > y and 0 when x < y. Once the CRPS is equals to 0, the forecast is wholly accurate, conversely, the higher the CRPS, the worse the performance of the forecast. 

This code automatically calculate two times (before, after bias correction).

Also, sometimes we can face the issue from the number of ensemble members. Most of originating centres have changed number of ensemble members once. (Please see A.Download seasonal forecasts datasets / 3. Seasonal forecasts systems and datasets for 8 originating centres / Total precipitation table). In this case, we need to designate exact location and number of ensemble manually. 

This example show you the case when we apply ECMWF datasets to calculate CRPS from 2011 to 2020. In this case, there are 25 ensemble members and 72 rows from Jan.2011 to Dec.2016, also rest of the data have 51 ensemble members. If the number of ensemble members is same, you can put the same number on it.  <font color='red'> Remember, you need to manually to revise the code below;</font>

In [17]:
# Before bias correction
bc_type = 'before'

# (Should be manually revised) 
num_row1 = 72    # The number of rows for the first datasets having 'num_col1' of ensemble members
num_col1 = 25    # The number of ensemble members for the first datasets
num_col2 = 51    # The number of ensemble members for the second datasets

# CRPS calculation
for leadtime in range(1,maxleadtime+1):
    df = pd.read_csv(calcpath + str(catch_name) + '_' + str(leadtime) + '_' + bc_type + '_bc.csv')
    df_a = df.to_numpy().astype(float)
    df_a2 = df_a[:,1:df_a.shape[1]-2]
    df_a3 = df_a2[:num_row1, :num_col1]     # data slicing
    df_a4 = df_a2[num_row1:, :num_col2]     # data slicing
    df_obs1 = df_a[:num_row1, num_col2+2]   # data slicing
    df_obs2 = df_a[num_row1:, num_col2+2]   # data slicing
        
    crps_dictionary_rand1 = em.ens_crps(df_obs1, df_a3)
    crps_dictionary_rand2 = em.ens_crps(df_obs2, df_a4)
    temp1 = crps_dictionary_rand1['crps']
    temp2 = crps_dictionary_rand2['crps']
    crps = np.concatenate([temp1, temp2], axis=0)
    csv = pd.DataFrame(crps)
    csv['month'] = df['date'].str.slice(start=5,stop=7)
    csv.set_index(df['date'], inplace=True)
    csv=csv[['month',0]]
    csv=csv.rename(columns={0:'CRPS_'+str(leadtime)})
    csv.to_csv(calcpath + 'CRPS_' + str(leadtime) + '_' + str(catch_name) +'_' + str(bc_type) + '_bc.csv')
leadtime = leadtime+1

# Data merge for whole lead time
base = pd.read_csv(calcpath + 'CRPS_' + str(1) + '_' + str(catch_name) +'_' + str(bc_type) + '_bc.csv')
base['leadtime'] = 1
base['CRPS'] = round(base['CRPS_1'],3)
base.drop(columns=['CRPS_1'], inplace=True)
for leadtime in range(2,maxleadtime+1):
    df = pd.read_csv(calcpath + 'CRPS_' + str(leadtime) + '_' + str(catch_name) +'_' + str(bc_type) + '_bc.csv')
    df['CRPS'] = round(df['CRPS_' + str(leadtime)],3)
    df.drop(columns=['CRPS_' + str(leadtime)], inplace=True)
    df['leadtime'] = leadtime        
    base = pd.concat([base, df])
leadtime = leadtime+1
base['catchment'] = str(catch_name)
base.set_index('catchment', inplace=True)
base.to_csv(calcpath + '[CRPS]_' + str(catch_name) +'_'  + str(originating_centre.lower()) + '_' + str(bc_type) + '_bc.csv')

#------------------------------------------------------------------------------------------------------------------------
# After bias correction
bc_type = 'after'

# CRPS calculation
for leadtime in range(1,maxleadtime+1):
    df = pd.read_csv(calcpath + str(catch_name) + '_' + str(leadtime) + '_' + bc_type + '_bc.csv')
    df_a = df.to_numpy().astype(float)
    df_a2 = df_a[:,1:df_a.shape[1]-2]
    df_a3 = df_a2[:num_row1, :num_col1]     # data slicing
    df_a4 = df_a2[num_row1:, :num_col2]     # data slicing
    df_obs1 = df_a[:num_row1, num_col2+2]   # data slicing
    df_obs2 = df_a[num_row1:, num_col2+2]   # data slicing
        
    crps_dictionary_rand1 = em.ens_crps(df_obs1, df_a3)
    crps_dictionary_rand2 = em.ens_crps(df_obs2, df_a4)
    temp1 = crps_dictionary_rand1['crps']
    temp2 = crps_dictionary_rand2['crps']
    crps = np.concatenate([temp1, temp2], axis=0)  
    csv = pd.DataFrame(crps)
    csv['month'] = df['date'].str.slice(start=5,stop=7)
    csv.set_index(df['date'], inplace=True)
    csv=csv[['month',0]]
    csv=csv.rename(columns={0:'CRPS_'+str(leadtime)})
    csv.to_csv(calcpath + 'CRPS_' + str(leadtime) + '_' + str(catch_name) +'_' + str(bc_type) + '_bc.csv')
leadtime = leadtime+1

# Data merge for whole lead time
base = pd.read_csv(calcpath + 'CRPS_' + str(1) + '_' + str(catch_name) +'_' + str(bc_type) + '_bc.csv')
base['leadtime'] = 1
base['CRPS'] = round(base['CRPS_1'],3)
base.drop(columns=['CRPS_1'], inplace=True)
for leadtime in range(2,maxleadtime+1):
    df = pd.read_csv(calcpath + 'CRPS_' + str(leadtime) + '_' + str(catch_name) +'_' + str(bc_type) + '_bc.csv')
    df['CRPS'] = round(df['CRPS_' + str(leadtime)], 3)
    df.drop(columns=['CRPS_' + str(leadtime)], inplace=True)
    df['leadtime'] = leadtime        
    base = pd.concat([base, df])
leadtime = leadtime+1
base['catchment'] = str(catch_name)
base.set_index('catchment', inplace=True)
base.to_csv(calcpath + '[CRPS]_' + str(catch_name) +'_'  + str(originating_centre.lower()) + '_' + str(bc_type) + '_bc.csv')

print('Successful')

Successful


### 4.1. Climatology CRPS calculation


#### 4.1 Rearrange Datasets by lead time (climatology)

To calculate CRPSS, you should calculate the CRPSS of climatology. Here, climatology represents the past observed precipitation. 
Once again, to calculate CRPS, we need to rearrange the datasets along with lead time. To run the code, firstly you should specify the earliest year that you have as climatology.

In [7]:
start_year_cli = 1994  # The earliest year that you have climatology data.

# Call the observed data (climatology) and basic treatment
leadtime=1
df = pd.read_csv('./assessment/obsdata_tp.csv')
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True, format='%m/%d/%Y', errors='ignore')
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df2=df.groupby(['year', 'month']).sum()
df2=df2.reset_index()
df2=df2[df2['year']<=start_year]
df2=df2.pivot_table(str(catch_name), ['month'], 'year')
df2=df2.reset_index()

df3 = pd.read_csv(calcpath + str(catch_name) + '_' + str(1) + '_before_bc.csv')

# Data rearranging work for 1 month of lead time
empty_df = pd.DataFrame()
empty_df['date'] = df3['date']
for i in range(start_year_cli, start_year):
    empty_df[i] = df2[i]

for j in range(12, empty_df.shape[0]):
    empty_df.iloc[j] = empty_df.iloc[j-12]
j=j+1
empty_df['mean'] = round(empty_df.iloc[:,1:empty_df.shape[1]+2].mean(axis=1),2)
empty_df['obs'] = df3['obs']
empty_df['date'] = df3['date']
empty_df.set_index('date', inplace=True)
empty_df.to_csv(calcpath + str(catch_name) + '_' + str(leadtime) +'_climatology.csv')

# Rearranging data for the other lead times
df_cli = pd.read_csv(calcpath + str(catch_name) + '_' + str(1) +'_climatology.csv')
for leadtime in range(2,maxleadtime+1):
    empty_df_cli = df_cli
    num_row = empty_df_cli.shape[0]
    for m in range(1,maxleadtime):
        empty_df_cli.loc[num_row+m] = empty_df_cli.loc[m-1]
    m = m+1
    for k in range(0, num_row):
        empty_df_cli.iloc[k] = empty_df_cli.iloc[k:k+leadtime].sum()
    k=k+1
    empty_df_cli=empty_df_cli.loc[:num_row]
    df_cli = pd.read_csv(calcpath + str(catch_name) + '_' + str(1) +'_climatology.csv')
    empty_df_cli['date'] = df_cli['date']
    empty_df_cli.set_index('date', inplace=True)
    empty_df_cli.to_csv(calcpath + str(catch_name) + '_' + str(leadtime) +'_climatology.csv')
leadtime=leadtime+1

print('Successful')

Successful


#### 4.2 CRPS calculation (climatology)

This code repeats the process 3.2 for climatology.

In [8]:
# CRPS calculation
for leadtime in range(1,maxleadtime+1):
    df = pd.read_csv(calcpath + str(catch_name) + '_' + str(leadtime) + '_climatology.csv')
    df_a = df.to_numpy().astype(float)
    df_a2 = df_a[:,1:df_a.shape[1]-2]
    df_a3 = df_a2[:, :]   
    df_obs1 = df_a[:, df_a.shape[1]-1]   # 0에서 72번 행까지 / 53번 컬럼값을 선택하기
    crps_dictionary_rand1 = em.ens_crps(df_obs1, df_a3)
    temp1 = crps_dictionary_rand1['crps']
    csv = pd.DataFrame(temp1)
    csv['month'] = df['date'].str.slice(start=5,stop=7)
    csv.set_index(df['date'], inplace=True)
    csv=csv[['month',0]]
    csv=csv.rename(columns={0:'CRPS_'+str(leadtime)})
    csv.to_csv(calcpath + 'CRPS_' + str(leadtime) + '_' + str(catch_name) +'_climatology.csv')
leadtime = leadtime+1

# Data merge for whole lead time
base = pd.read_csv(calcpath + 'CRPS_' + str(1) + '_' + str(catch_name) +'_climatology.csv')
base['leadtime'] = 1
base['CRPS_ref'] = base['CRPS_1']
base.drop(columns=['CRPS_1'], inplace=True)
for leadtime in range(2,maxleadtime+1):
    df = pd.read_csv(calcpath + 'CRPS_' + str(leadtime) + '_' + str(catch_name) +'_climatology.csv')
    df['CRPS_ref'] = round(df['CRPS_' + str(leadtime)],3)
    df.drop(columns=['CRPS_' + str(leadtime)], inplace=True)
    df['leadtime'] = leadtime        
    base = pd.concat([base, df])
leadtime = leadtime+1
base['catchment'] = str(catch_name)
base.set_index('catchment', inplace=True)
base.to_csv(calcpath + '[CRPS]_' + str(catch_name) +'_'  + str(originating_centre.lower()) + '_climatology.csv')

print('Successful')

Successful


### 5. CRPSS calculation

CRPSS compares the skill of seasonal forecasts with climatology, thus finally it can be simply calculated as 

$$ CRPSS=\ 1\ -\ \frac{{\rm CRPS}^{Sys}}{{\rm CRPS}^{Ref}}$$

where $CRPS^{Sys}$ is previously calculated $CRPS$ (seasonal forecasts), $CRPS^{Ref}$ represents the reference $CRPS$ obtained from climatology. When the skill score is higher (lower) than zero, the forecasting system is more (less) skilful than reference. When it is equal to zero, the system (seasonal forecasts) and the reference (Climatology) have equivalent skill. 

CRPSS can be calculated by runing the code below;

In [9]:
# Before bias correction
bc_type = 'before'

df = pd.read_csv(calcpath + '[CRPS]_'+ str(catch_name)  + '_' + str(originating_centre.lower()) + '_'  + str(bc_type) + '_bc.csv')
df_ref = pd.read_csv(calcpath + '[CRPS]_'+ str(catch_name)  + '_' + str(originating_centre.lower()) + '_climatology.csv')
df['CRPS_ref'] = round(df_ref['CRPS_ref'],3)
df['CRPSS'] = round(1 - df['CRPS'] / df['CRPS_ref'],3)
df.set_index('catchment', inplace=True)
df.to_csv(calcpath + '[CRPSS]_' + str(catch_name) + '_' + str(originating_centre.lower()) + '_'  + str(bc_type) + '_bc.csv')

#-----------------------------------------------------------------------------------------------------------------------------
# After bias correction
bc_type = 'after'

df = pd.read_csv(calcpath + '[CRPS]_'+ str(catch_name)  + '_' + str(originating_centre.lower()) + '_'  + str(bc_type) + '_bc.csv')
df_ref = pd.read_csv(calcpath + '[CRPS]_'+ str(catch_name)  + '_' + str(originating_centre.lower()) + '_climatology.csv')
df['CRPS_ref'] = round(df_ref['CRPS_ref'],3)
df['CRPSS'] = round(1 - df['CRPS'] / df['CRPS_ref'],3)
df.set_index('catchment', inplace=True)
df.to_csv(calcpath + '[CRPSS]_' + str(catch_name) + '_' + str(originating_centre.lower()) + '_'  + str(bc_type) + '_bc.csv')

print('Successful')

Successful
